# Assignment 2: Building a Simple Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [1]:
Summaries_file = 'data/influenza_Summaries.pkl.bz2'
Abstracts_file = 'data/influenza_Abstracts.pkl.bz2'

In [2]:
import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )
    
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for our example paper:

In [3]:
Summaries[28953867]

paper(title='Massively parallel de novo protein design for targeted therapeutics.', authors=['Chevalier A', 'Silva DA', 'Rocklin GJ', 'Hicks DR', 'Vergara R', 'Murapa P', 'Bernard SM', 'Zhang L', 'Lam KH', 'Yao G', 'Bahl CD', 'Miyashita SI', 'Goreshnik I', 'Fuller JT', 'Koday MT', 'Jenkins CM', 'Colvin T', 'Carter L', 'Bohn A', 'Bryan CM', 'Fernández-Velasco DA', 'Stewart L', 'Dong M', 'Huang X', 'Jin R', 'Wilson IA', 'Fuller DH', 'Baker D'], year=2017, doi='10.1038/nature23912')

In [4]:
Abstracts[28953867]

'De novo protein design holds promise for creating small stable proteins with shapes customized to bind therapeutic targets. We describe a massively parallel approach for designing, manufacturing and screening mini-protein binders, integrating large-scale computational design, oligonucleotide synthesis, yeast display screening and next-generation sequencing. We designed and tested 22,660 mini-proteins of 37-43 residues that target influenza haemagglutinin and botulinum neurotoxin B, along with 6,286 control sequences to probe contributions to folding and binding, and identified 2,618 high-affinity binders. Comparison of the binding and non-binding design sets, which are two orders of magnitude larger than any previously investigated, enabled the evaluation and improvement of the computational model. Biophysical characterization of a subset of the binder designs showed that they are extremely stable and, unlike antibodies, do not lose activity after exposure to high temperatures. The de

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive. We will improve them in a later assignment.

In [5]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Lorem ipsum dolor sit AMET")))

['lorem', 'ipsum', 'dolor', 'sit', 'amet']


In [6]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(28953867)
display_summary(28953867, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the abstracts of the papers in our dataset.

We will implement our inverted index as a **Python dictionary with terms as keys and posting lists as values**. For the posting lists, instead of using Python lists and then implementing the different operations on them ourselves, we will use **Python sets** and use the predefined set operations to process these posting "lists". This will also ensure that each document is added at most once per term. The use of Python sets is not the most efficient solution but will work for our purposes. (As an optional additional exercise, you can try to implement the posting lists as Python lists for this and the following assignments.)

Not every paper in our dataset has an abstract; we will only index papers for which an abstract is available.

In [7]:
from collections import defaultdict

inverted_index = defaultdict(set)

# This may take a while:
for (id, abstract) in Abstracts.items():
    for term in preprocess(tokenize(abstract)):
        inverted_index[term].add(id)

Let's see what's in the index for the example term 'madagascar':

In [8]:
print(inverted_index['madagascar'])

{12458917, 23169961, 22814442, 12631982, 25842000, 20092624, 23169972, 21444983, 15678809, 15678810, 24893021}


We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'rotterdam':

In [9]:
query_word = 'rotterdam'
for i in inverted_index[query_word]:
    display_summary(i)

----------

# Tasks

**Your name:** David Rocker

### Task 1

Construct a function called `and_query` that takes as input a single string, consisting of one or more words, such that the function returns a list of matching documents. `and_query`, as its name suggests, should require that all query terms are present in the documents of the result list. Demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

(You can use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query. You can also exploit the fact that the posting lists are [sets](https://docs.python.org/3/library/stdtypes.html#set), which means you can easily perform set operations such as union, difference and intersect on them.)

In [10]:
def and_query(searchString):
    searchWords = preprocess(tokenize(searchString)) 
    results = set()
    for word in searchWords:
        currWordDocuments = inverted_index[word]
        if len(results) == 0:
            results = currWordDocuments
        else:
            results = results & currWordDocuments
    return results

for i in and_query("Prevalence and Predictors"):
        display_summary(i)


### Task 2

Construct a second function called `or_query` that works in the same way as `and_query` you just implemented, but returns as function value the documents that contain _at least one_ of the words in the query. Demonstrate the working of this second function also with an example (again, choose one that leads to fewer than 100 hits).

In [11]:
def or_query(searchString):
    searchWords = preprocess(tokenize(searchString)) 
    results = set()
    for word in searchWords:
        currWordDocuments = inverted_index[word]
        if len(results) == 0:
            results = currWordDocuments
        else:
            results = results | currWordDocuments
    return results

for i in or_query("Forward Blown"):
        display_summary(i)


### Task 3

Show how many hits the query "to be or not to be" returns for your two query functions (`and_query` and `or_query`).

In [12]:
query_string = "to be or not to be"
print("Or:")
print(len(or_query(query_string)))
print("\nAnd:")
print(len(and_query(query_string)))

Or:
62070

And:
4267


### Task 4

Given the nature of our dataset, how many of the documents from task 3 do you think are actually about the [Shakespeare quote](https://en.wikipedia.org/wiki/To_be%2C_or_not_to_be)? What could you do to better handle such queries? (You don't have to implement this yet!)

**Answer:** I do not think any of the documents are about the Shakespeare quote. To better handle such queries you could implement biword queries, which is susceptible to false positives, or use positional indexes to find the quote in its entirety. 

### Task 5

Why does `and_query('therapeutic protections antibodies')` not return our example paper 28953867, even though it mentions antibodies and therapeutic protections in the abstract? (You do not have to implement anything to fix this yet!)

**Answer:** It is a false negative due to the lack of lemmatization

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done individually and group submissions are **not allowed**.